<a href="https://colab.research.google.com/github/cagBRT/CANBus/blob/main/CANBUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -l -s https://github.com/cagBRT/CANBus.git cloned-repo
%cd cloned-repo
!ls

Program to compare two CAN Bus output files

In [ ]:
import re

Classes

In [ ]:
class CANLine:
    def __init__(self, time, channel, messID, pld):
        self.time = time
        self.channel = channel
        self.messID = messID
        self.pld = pld

In [ ]:
class MessageObj:
    def __init__(self, mID, pLD):
        self.mID = mID
        self.pLD = pLD

Helper functions

In [ ]:
def procCAN(line):
    m = re.compile(
        '(?P<Time>[+-]?\d*\.?\d+)[ ]+(?P<BusID>\d+)[ ]+(?P<MessageID>[0-9A-Fa-f]+)(?P<ExtendedFlag>x|)[ ]+(?P<Transceive>[RT]x)[ ]+d[ ]+(?P<ByteLength>\d+)(?P<Data>(?: [0-9A-Fa-f]{2})+).*')
    g = m.search(line)
    if g:
        time = g.group(1)
        channel = g.group(2)
        messID = (g.group(3) + g.group(4))
        pld = g.group(7)
        canList.append(CANLine(time, channel, messID, pld))

Read the file into a list called canList

In [ ]:
#You must request access to this file from catherine.gamboa@bluerivertech.com
#You must be a BRT or JD employee
file = 'Loggerc4-00-ad-16-95-7a_2020-09-06_180558_00066_can_split_00001.asc'

In [ ]:
canList = list()
trueBus = list()  # VehB1
spoofBus = list() # HAD2

In [ ]:
with open(file, 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        else:
            procCAN(line)

In [ ]:
# List contains every CAN line as an object
#   This is also working for lines that have a data payload of 3
print("The length of the list is: " + str(len(canList)))
#The structure of the data is
#Time Channel MessageID PLD
#For example row 244 has the following values
print("Time column: ",canList[244].time)
print("Channel column: ",canList[244].channel)
print("Message ID column: ",canList[244].messID)
print("Payload Column:",canList[244].pld)


In [ ]:
# ****************************************************** Time Comparison ******************************************************************************************
# Create a list of times from each CAN Channel of interest and then 
#finds the time difference which will be added to a list and 
#then calculate the average of diff list

for item in canList:
    if item.channel == '20':
        trueBus.append(item)
    elif item.channel == '23':
        spoofBus.append(item)
    else:
        pass

# It looks like spoof bus missed one single message from VehB1 but that could be from right where I cut off the log potentiially
#   I was never able to run a good count non-programmatically on the file to verify these numbers but I am confident
#print('Found ' + str(trueBus.__len__()) + ' items for VehB1') # 6370
#print('Found ' + str(spoofBus.__len__()) + ' items for HAD2') # 6369

tbTimes = list()
sbTimes = list()
timeDiff = list()

for item in trueBus:
    tbTimes.append(item.time)
for item in spoofBus:
    sbTimes.append(item.time)

for t, s in zip(tbTimes, sbTimes):
    difference = float(s) - float(t)
    timeDiff.append(difference)

# Verified that timeDiff does have 6369 elements
timeDiffAVG = sum(timeDiff) / len(timeDiff)



# ****************************************************** Copy Integrity and Outliers *********************************************

# Need to do Message ID and Payload


tbM = list()
sbM = list()

for item in trueBus:
    tbM.append(MessageObj(item.messID, item.pld))
for item in spoofBus:
    sbM.append(MessageObj(item.messID, item.pld))

errorCount = 0

for t, s in zip(tbM, sbM):
    if t.mID == s.mID and t.pLD == s.pLD:
        print("************** Success ********************")
        print("Spoof Info")
        print(s.mID)
        print(s.pLD)
        print("True Info")
        print(t.mID)
        print(t.pLD)
        print('*****************************************')
    else:
        errorCount += 1
        print("************** Error ********************")
        print("Spoof Info")
        print(s.mID)
        print(s.pLD)
        print("True Info")
        print(t.mID)
        print(t.pLD)
        print('*****************************************')

print("***************************************Final Results***********************************************************")
print("The average time difference between the spoofed bus and the production bus is: " + str(timeDiffAVG) + ' seconds')
print(str(errorCount) + ' messages were seen as not a direct copy')
messCountDiff = trueBus.__len__() - spoofBus.__len__()
print(str(messCountDiff) + ' message was found on the production bus that was not spoofed to the instrumentation bus')



